https://imsdb.com/scripts/Wolf-of-Wall-Street,-The.html

https://towardsdatascience.com/speech-recognition-with-timestamps-934ede4234b2

## Text is taken from the script of ***The Wolf of Wall Street*** movie and is processed from text to speech with **Murf.AI** as mono .wav files

## Text 1
*You want to know what money sounds like? Visit a trading floor on Wall Street. F\*ck this, sh\*t that. C\*ck, c\*nt, \*sshole. I couldn't believe how these guys talked to each other*

## Text 2
But why should you give a sh\*t? You're not the one who researched the f\*cking thing and dealt with the f\*cking golf course people!

## Text 3
I'm a d\*chebag?!! Well, whose f\*cking idea was it made us all this money?!! Who f\*cking knew Steve Madden?!! This d\*chebag!! So go f\*ck yourself a little!

## Text 4
What I'm asking, you Swiss d\*ck, is are you going to f\*ck me over. I understand perfectly, you American sh\*theel.

## Text 5
Even though I owned 85% of Steve-C\*cksucker-Madden-f\*cking-Shoes, the shares were in his f\*cking name. Motherf\*cker knew I was in trouble with the Feds and was trying to take advantage.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pydub

!pip install vosk

!pip install better-profanity

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 KB 3.4 MB/s eta 0:00:00


In [ ]:
!wget -nc https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip

!unzip vosk-model-en-us-0.22.zip

--2023-04-02 11:40:54--  https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1913365522 (1.8G) [application/zip]
Saving to: ‘vosk-model-en-us-0.22.zip’

vosk-model-en-us-0. 100%[===================>]   1.78G  19.4MB/s    in 93s     

2023-04-02 11:42:27 (19.6 MB/s) - ‘vosk-model-en-us-0.22.zip’ saved [1913365522/1913365522]

Archive:  vosk-model-en-us-0.22.zip
   creating: vosk-model-en-us-0.22/
   creating: vosk-model-en-us-0.22/am/
  inflating: vosk-model-en-us-0.22/am/final.mdl  
  inflating: vosk-model-en-us-0.22/am/tree  
   creating: vosk-model-en-us-0.22/ivector/
  inflating: vosk-model-en-us-0.22/ivector/final.dubm  
  inflating: vosk-model-en-us-0.22/ivector/final.ie  
  inflating: vosk-model-en-us-0.22/ivector/final.mat  
  inflating: vosk-model-en-us-0

In [ ]:
import wave
import json
from pydub import AudioSegment
from IPython.display import display, Audio

from vosk import Model, KaldiRecognizer, SetLogLevel
from better_profanity import profanity

In [ ]:
audio_folder = "/content/drive/MyDrive/Datasets/profanity_censor_test"
model_path = "/content/vosk-model-en-us-0.22"

In [ ]:
class Word:
    def __init__(self, dict):
        self.conf = dict["conf"]
        self.end = dict["end"]
        self.start = dict["start"]
        self.word = dict["word"]

    def to_string(self):
        return "{:20} from {:.2f} sec to {:.2f} sec, confidence is {:.2f}%".format(
            self.word, self.start, self.end, self.conf*100)

In [ ]:
def detect_audio(audio_path):
  model = Model(model_path)
  wf = wave.open(audio_path, "rb")
  rec = KaldiRecognizer(model, wf.getframerate())
  rec.SetWords(True)

  results = []

  while True:
      data = wf.readframes(4000)
      if len(data) == 0:
          break
      if rec.AcceptWaveform(data):
          part_result = json.loads(rec.Result())
          results.append(part_result)
  part_result = json.loads(rec.FinalResult())
  results.append(part_result)

  list_of_Words = []

  for sentence in results:
      if len(sentence) == 1:
          continue
      for obj in sentence['result']:
          w = Word(obj)
          list_of_Words.append(w)

  wf.close()

  # for word in list_of_Words:
  #   print(word.to_string())

  return list_of_Words

In [ ]:
bleep = AudioSegment.from_wav("/content/drive/MyDrive/Datasets/profanity_censor_test/bleep.wav")

def bleep_audio(audio, start, end):
  before_segment = audio[:start]
  after_segment = audio[end:]
  segment_length = end - start

  bleep_segment =  bleep[:segment_length]

  return before_segment + bleep_segment + after_segment
  

In [ ]:
audio_test = "/content/drive/MyDrive/Datasets/profanity_censor_test/SRTest1.wav"

word_list = detect_audio(audio_test)

audio_signal = AudioSegment.from_wav(audio_test)

for word in word_list:
  if profanity.contains_profanity(word.word):
    audio_signal = bleep_audio(audio_signal, word.start*1000, word.end*1000)

audio_signal.export("test1.wav")
display(Audio("/content/test1.wav"))

you                  from 0.42 sec to 0.60 sec, confidence is 100.00%
want                 from 0.60 sec to 0.84 sec, confidence is 100.00%
to                   from 0.84 sec to 0.93 sec, confidence is 100.00%
know                 from 0.93 sec to 1.14 sec, confidence is 100.00%
what                 from 1.14 sec to 1.35 sec, confidence is 100.00%
money                from 1.35 sec to 1.65 sec, confidence is 100.00%
sounds               from 1.65 sec to 1.98 sec, confidence is 100.00%
like                 from 1.98 sec to 2.34 sec, confidence is 100.00%
visit                from 3.21 sec to 3.60 sec, confidence is 100.00%
a                    from 3.60 sec to 3.66 sec, confidence is 100.00%
trading              from 3.66 sec to 4.11 sec, confidence is 100.00%
floor                from 4.11 sec to 4.44 sec, confidence is 100.00%
on                   from 4.44 sec to 4.59 sec, confidence is 100.00%
wall                 from 4.59 sec to 4.80 sec, confidence is 100.00%
street              

In [ ]:
audio_test = "/content/drive/MyDrive/Datasets/profanity_censor_test/SRTest2.wav"

word_list = detect_audio(audio_test)

audio_signal = AudioSegment.from_wav(audio_test)

for word in word_list:
  if profanity.contains_profanity(word.word):
    audio_signal = bleep_audio(audio_signal, word.start*1000, word.end*1000)

audio_signal.export("test2.wav")
display(Audio("/content/test2.wav"))

but                  from 0.87 sec to 1.05 sec, confidence is 100.00%
why                  from 1.05 sec to 1.29 sec, confidence is 100.00%
should               from 1.29 sec to 1.50 sec, confidence is 100.00%
you                  from 1.50 sec to 1.62 sec, confidence is 100.00%
give                 from 1.62 sec to 1.86 sec, confidence is 100.00%
a                    from 1.86 sec to 1.92 sec, confidence is 100.00%
shit                 from 1.92 sec to 2.34 sec, confidence is 100.00%
you're               from 3.24 sec to 3.45 sec, confidence is 100.00%
not                  from 3.45 sec to 3.66 sec, confidence is 100.00%
the                  from 3.66 sec to 3.75 sec, confidence is 100.00%
one                  from 3.75 sec to 3.96 sec, confidence is 100.00%
who                  from 3.96 sec to 4.08 sec, confidence is 100.00%
researched           from 4.08 sec to 4.59 sec, confidence is 83.13%
the                  from 4.59 sec to 4.68 sec, confidence is 100.00%
fucking              

In [ ]:
audio_test = "/content/drive/MyDrive/Datasets/profanity_censor_test/SRTest3.wav"

word_list = detect_audio(audio_test)

audio_signal = AudioSegment.from_wav(audio_test)

for word in word_list:
  if profanity.contains_profanity(word.word):
    audio_signal = bleep_audio(audio_signal, word.start*1000, word.end*1000)

audio_signal.export("test3.wav")
display(Audio("/content/test3.wav"))

i'm                  from 0.75 sec to 0.96 sec, confidence is 100.00%
a                    from 0.96 sec to 1.02 sec, confidence is 100.00%
douche               from 1.02 sec to 1.35 sec, confidence is 100.00%
bag                  from 1.38 sec to 1.80 sec, confidence is 100.00%
well                 from 2.61 sec to 2.97 sec, confidence is 100.00%
whose                from 3.03 sec to 3.30 sec, confidence is 100.00%
fucking              from 3.30 sec to 3.63 sec, confidence is 100.00%
idea                 from 3.63 sec to 4.02 sec, confidence is 100.00%
was                  from 4.02 sec to 4.20 sec, confidence is 100.00%
it                   from 4.20 sec to 4.29 sec, confidence is 100.00%
made                 from 4.29 sec to 4.50 sec, confidence is 100.00%
us                   from 4.50 sec to 4.65 sec, confidence is 100.00%
all                  from 4.65 sec to 4.80 sec, confidence is 100.00%
this                 from 4.80 sec to 4.98 sec, confidence is 100.00%
money               

In [ ]:
audio_test = "/content/drive/MyDrive/Datasets/profanity_censor_test/SRTest4.wav"

word_list = detect_audio(audio_test)

audio_signal = AudioSegment.from_wav(audio_test)

for word in word_list:
  if profanity.contains_profanity(word.word):
    audio_signal = bleep_audio(audio_signal, word.start*1000, word.end*1000)

audio_signal.export("test4.wav")
display(Audio("/content/test4.wav"))

what                 from 0.96 sec to 1.17 sec, confidence is 100.00%
i'm                  from 1.17 sec to 1.32 sec, confidence is 100.00%
asking               from 1.32 sec to 1.80 sec, confidence is 100.00%
you                  from 1.86 sec to 2.04 sec, confidence is 100.00%
swiss                from 2.04 sec to 2.37 sec, confidence is 100.00%
dick                 from 2.37 sec to 2.70 sec, confidence is 100.00%
is                   from 2.79 sec to 2.97 sec, confidence is 100.00%
are                  from 2.97 sec to 3.09 sec, confidence is 100.00%
you                  from 3.09 sec to 3.24 sec, confidence is 100.00%
going                from 3.24 sec to 3.48 sec, confidence is 100.00%
to                   from 3.48 sec to 3.57 sec, confidence is 100.00%
fuck                 from 3.57 sec to 3.81 sec, confidence is 100.00%
me                   from 3.81 sec to 3.99 sec, confidence is 100.00%
over                 from 3.99 sec to 4.35 sec, confidence is 100.00%
i                   

In [ ]:
audio_test = "/content/drive/MyDrive/Datasets/profanity_censor_test/SRTest5.wav"

word_list = detect_audio(audio_test)

audio_signal = AudioSegment.from_wav(audio_test)

for word in word_list:
  if profanity.contains_profanity(word.word):
    audio_signal = bleep_audio(audio_signal, word.start*1000, word.end*1000)

audio_signal.export("test5.wav")
display(Audio("/content/test5.wav"))

even                 from 0.45 sec to 0.72 sec, confidence is 100.00%
though               from 0.72 sec to 0.90 sec, confidence is 100.00%
i                    from 0.90 sec to 1.02 sec, confidence is 100.00%
owned                from 1.02 sec to 1.23 sec, confidence is 100.00%
eighty               from 1.26 sec to 1.50 sec, confidence is 100.00%
five                 from 1.50 sec to 1.77 sec, confidence is 100.00%
percent              from 1.77 sec to 2.19 sec, confidence is 100.00%
of                   from 2.19 sec to 2.28 sec, confidence is 100.00%
steve                from 2.28 sec to 2.58 sec, confidence is 100.00%
cox                  from 2.58 sec to 2.85 sec, confidence is 89.21%
or                   from 2.85 sec to 2.94 sec, confidence is 78.12%
chromatin            from 2.94 sec to 3.36 sec, confidence is 89.21%
fucking              from 3.36 sec to 3.72 sec, confidence is 100.00%
shoes                from 3.72 sec to 4.08 sec, confidence is 100.00%
the                  fr